In [ ]:
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. 
Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. 
Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: 
    выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

In [1]:
# подключение библиотек

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from catboost import Pool, CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import preprocessing

In [2]:
# загрузка даннных

data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


In [3]:
# предобработка данных

def data_preprocess (df):
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
    for l in df["Product_Info_2_1"].unique():
        df["Product_Info_2_1" + l] = df["Product_Info_2_1"].isin([l]).astype("int8")
    df.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
    df.fillna(value=-1, inplace=True)
    data["Response"] = data["Response"] - 1
    return df

In [4]:
data = data_preprocess(data)

In [5]:
# набор столбцов для расчетов

columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

In [6]:
# нормализация данных

scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

In [7]:
# оптимизация памяти

def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [8]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


In [9]:
x = pd.DataFrame(data_transformed, columns=columns_transformed)

In [10]:
# построение модели XGBoost

model_xgb = XGBClassifier(max_depth=17, max_features=27,
                      n_estimators=76, min_samples_leaf=20)
model_xgb.fit(x, data['Response'])

[11:44:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_features=27,
              max_leaves=None, min_child_weight=None, min_samples_leaf=20,
              missing=nan, monotone_constraints=None, n_estimators=76,
              n_jobs=None, num_parallel_tree=None, ...)

In [11]:
# построение моделей CatBoost


model_cb = CatBoostClassifier(iterations=10000, learning_rate=0.57,
            random_seed=17, depth=6, l2_leaf_reg=2,
            loss_function='MultiClass', bootstrap_type="MVS")
model_cb.fit(Pool(data=x, label=data["Response"]))

0:	learn: 1.5330128	total: 213ms	remaining: 35m 25s
1:	learn: 1.4359398	total: 266ms	remaining: 22m 7s
2:	learn: 1.3427639	total: 319ms	remaining: 17m 42s
3:	learn: 1.3129551	total: 375ms	remaining: 15m 36s
4:	learn: 1.2920150	total: 431ms	remaining: 14m 21s
5:	learn: 1.2754757	total: 486ms	remaining: 13m 29s
6:	learn: 1.2629586	total: 531ms	remaining: 12m 37s
7:	learn: 1.2538166	total: 578ms	remaining: 12m 1s
8:	learn: 1.2443132	total: 625ms	remaining: 11m 33s
9:	learn: 1.2374051	total: 673ms	remaining: 11m 12s
10:	learn: 1.2335046	total: 720ms	remaining: 10m 54s
11:	learn: 1.2294222	total: 773ms	remaining: 10m 43s
12:	learn: 1.2236002	total: 820ms	remaining: 10m 29s
13:	learn: 1.2143246	total: 870ms	remaining: 10m 20s
14:	learn: 1.2118135	total: 916ms	remaining: 10m 10s
15:	learn: 1.2085828	total: 963ms	remaining: 10m 1s
16:	learn: 1.2040890	total: 1.01s	remaining: 9m 56s
17:	learn: 1.1996522	total: 1.06s	remaining: 9m 51s
18:	learn: 1.1974595	total: 1.11s	remaining: 9m 44s
19:	learn

In [12]:
# построение модели градиентный бустинг

model_gbc = GradientBoostingClassifier(random_state=17, max_depth=13,
                max_features=26, min_samples_leaf=21, n_estimators=75)
model_gbc.fit(x, data['Response'])

GradientBoostingClassifier(max_depth=13, max_features=26, min_samples_leaf=21,
                           n_estimators=75, random_state=17)

In [13]:
# построение модели LightGBM

model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
    min_child_samples=17, num_leaves=35, n_estimators=10000)
model_lgb.fit(x, data['Response'])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012022 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2467
[LightGBM] [Info] Number of data points in the train set: 59381, number of used features: 118
[LightGBM] [Info] Start training from score 4.636837


LGBMRegressor(max_depth=18, min_child_samples=17, n_estimators=10000,
              num_leaves=35, random_state=17)

In [14]:
# загрузка тестовых данных для расчета

data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
data_test = data_preprocess(data_test)
data_test = reduce_mem_usage(data_test)
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                                     columns=columns)))
print (data_test_transformed.info())

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float32(118)
memory usage: 8.9 MB
None


In [36]:
# расчет предсказаний для каждой модели

data_test["target_xgb"] = model_xgb.predict(data_test_transformed)
data_test_copied = data_test_transformed.copy()

In [37]:
data_test["target_cb"] = model_cb.predict(Pool(data=data_test_transformed))
data_test_copied = data_test_transformed.copy()

In [18]:
data_test["target_gbc"] = model_gbc.predict(data_test_transformed)
data_test_copied = data_test_transformed.copy()

In [35]:
data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")
data_test_copied = data_test_transformed.copy()
print(data_test_copied.head())

        0         1         2         3         4         5         6    \
0  0.519789  1.002921  1.045952  0.022140  0.611857 -0.169414 -1.159587   
1  0.215425  0.266273  1.122714  0.126020 -1.634368 -0.169414  0.862391   
2  0.308653  0.022915  0.894903  0.405695  0.611857 -0.169414 -1.159587   
3 -0.278139 -0.707156  0.592804  0.143999 -1.634368 -0.169414  0.862391   
4 -0.513953 -0.463799 -0.542540 -0.333447  0.611857 -0.169414 -1.159587   

        7         8         9    ...       108       109       110       111  \
0  1.101046 -1.156735  1.130555  ...  0.559558 -0.083689  0.441621 -0.149284   
1 -1.013721  0.864261 -0.928723  ... -0.892015 -0.083689  0.441621 -0.149284   
2  1.101046 -1.156735  1.130555  ... -0.652247 -0.083689  0.441621 -0.149284   
3 -1.013721  0.862242  0.100916  ... -0.627190 -0.083689 -2.264385 -0.149284   
4  1.101046 -1.156735  1.130555  ... -0.892015 -0.083689  0.441621 -0.149284   

        112       113       114       115       116       117  
0 -0

In [ ]:
# Классы смещены на 1: начинаются от 0 и заканчиваются 7. Судя по рассчитанным матрицам ошибок, 
# для 0, 1, 3, 4 и 6 классов точнее работает градиентный бустинг, для 2 - XGBoost, для 5 - LightGBM, 
# для 7 - логистическая регрессия.


In [40]:
data_test_copied = data_transformed.copy()
x_copied = pd.DataFrame(data_test_copied, columns=columns_transformed)
copied_dataset = Pool(data=x_copied, label=data_test_copied["Response"])
data_test_copied["target_xgb"] = model_xgb.predict(x_copied)
data_test_copied["target_cb"] = model_cb.predict(copied_dataset)
data_test_copied["target_gbc"] = model_gbc.predict(x_copied)
data_test_copied["target_lgb"] = np.round(model_lgb.predict(x_copied)).astype("int8")

In [41]:
def vote_class (x):
    #если класс 2 то мы используем XGBoost
    if x.target_xgb == 2:
        class_ = x.target_xgb
    #Если класс 7, то мы используем LightGBM 
    elif x.target_lgb == 7:
        class_ = x.target_lgb
    #Если класс 0, то мы используем CatBoost
    elif x.target_cb == 0:
        class_ = x.target_cb
    #Во всех остальных случаях Градиентный бустинг
    else:
        class_ = x.target_gbc
    #в серию даннхы "Response" записываем результат class_ увеличенный на один
    x["Response"] = class_ + 1
    return x

In [42]:
# Применяем функцию перекрёстной проверки

data_test_copied = data_test_copied.apply(vote_class, axis=1)
print (data_test_copied.head())

          0         1         2         3         4         5         6  \
0 -1.618164 -1.690430  1.198242 -1.198242 -1.634766 -0.169434  0.862305   
1 -1.805664 -1.445312 -1.753906 -1.613281  0.611816 -0.169434  0.862305   
2 -0.043610  0.514160 -1.905273 -0.332764  0.611816 -0.169434 -1.159180   
3 -0.983398 -0.465576 -1.224609 -0.957520  0.611816 -0.169434 -1.159180   
4 -0.654297 -0.710449  0.062622 -0.371582  0.611816 -0.169434 -1.159180   

          7         8         9  ...       113       114       115       116  \
0 -1.013672  0.862305 -0.928711  ...  0.750977 -0.623535 -0.215942 -0.128906   
1 -1.013672  0.861328 -0.928711  ... -1.332031  1.604492 -0.215942 -0.128906   
2  1.100586 -1.156250  1.130859  ... -1.332031 -0.623535  4.628906 -0.128906   
3  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   
4  1.100586 -1.156250  1.130859  ...  0.750977 -0.623535 -0.215942 -0.128906   

       117  Response  target_xgb  target_cb  target_gbc  target_lgb 

In [44]:
# выгрузите результат в виде submission.csv

submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
submission["Response"] = data_test_copied["Response"].astype("int8")
submission.to_csv("submission.csv", index=False)
print (len(submission["Response"]) + 1)

19766


In [45]:
#создаём список class_target, в котором 8 текстовых элементов ["target_gbc"]
class_target = ["target_gbc"]*8
#Создаём функцию vote_class_enumerate
def vote_class_enumerate (x):
    #обходим циклом список class_target
    #получаем индексы и знчения строк
    for _,target in enumerate(class_target):
        #если во фрейме находим серию данных с названием равным target
        if x[target] == _:
            #то присваеваем это значение серии данных Response
            x["Response"] = x[target]
            #выходим из цикла
            break
    #возвращаем x
    return x

In [47]:
#Создаём переменную kappa_min

kappa_min = 0
for target_model in ["xgb", "cb", "gbc", "lgb"]:
    print ("Проверяем модель:", target_model)
    target_model = "target_" + target_model
    for class_ in range(0,8):
        target_model_prev = class_target[class_]
        class_target[class_] = target_model
        data_test_copied = data_test_copied.apply(vote_class_enumerate, axis=1)
        kappa = cohen_kappa_score(data_test_copied["Response"], 
                data["Response"], weights='quadratic')
        if kappa > kappa_min:
            kappa_min = kappa
        else:
            class_target[class_] = target_model_prev
    print ("Максимальная оценка:", kappa_min)
print (class_target)

Проверяем модель: xgb
Максимальная оценка: 0.91923523748714
Проверяем модель: cb
Максимальная оценка: 0.923490383367688
Проверяем модель: gbc
Максимальная оценка: 0.9235305740712393
Проверяем модель: lgb
Максимальная оценка: 0.9239003098498227
['target_cb', 'target_cb', 'target_xgb', 'target_xgb', 'target_gbc', 'target_cb', 'target_lgb', 'target_xgb']


In [48]:
data_test_copied = data_test_copied.apply(vote_class_enumerate, axis=1)

In [49]:
print ("Результат:",
       round(cohen_kappa_score(data_test_copied["Response"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_test_copied["Response"], data["Response"]))

Результат: 0.924
[[    0     0     0     0     0     0     0     0     0]
 [ 6207     0     0     0     0     0     0     0     0]
 [    0  6552     0     0     0     0     0     0     0]
 [    0     0  1013     0     0     0     0     0     0]
 [    0     0     0  1428     6     0     0     0     0]
 [    0     0     0     0  5308    89    18    10     0]
 [    0     0     0     0   117 11143   202     0     0]
 [    0     0     0     0     1     0  7788   863     0]
 [    0     0     0     0     0     1    19 18616     0]]
